In [1]:
#Load Documents
import csv
import string
import sys

csv.field_size_limit(sys.maxsize)

train_raw_neg = []
train_raw_pos = []

with open('data/yelp/sentiment.train.0', 'rb') as fp:
    train_raw_neg = fp.readlines()
    
with open('data/yelp/sentiment.train.1', 'rb') as fp:
    train_raw_pos = fp.readlines()



In [2]:
neg_sent_cnt = len(train_raw_neg)
pos_sent_cnt = len(train_raw_pos)
print neg_sent_cnt
print pos_sent_cnt

177218
266041


In [3]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import re


stop_word_list = list(stopwords.words('english'))

def tokenize_std(content, n_gram):
    content = content.lower()
    content = content.replace("\\n", " ")
    words = content.split()
    tokens = []
    for i in range(n_gram):
        for j in range(len(words)-i):
            tokens.append("_".join(words[j:j+i+1]))
    
    tokens = [w for w in tokens if not w in stop_word_list]
    return tokens


In [4]:
train_token_neg = []
train_token_pos = []

for s in train_raw_neg:
    train_token_neg.append(tokenize_std(s, 2))

for s in train_raw_pos:
    train_token_pos.append(tokenize_std(s, 2))
    

In [5]:
from nltk import ngrams
from collections import Counter

unigram_neg = Counter([])
unigram_pos = Counter([])
unigram_all = Counter([])

bigram_neg = Counter([])
bigram_pos = Counter([])
bigram_all = Counter([])


for s in train_token_neg:
    for w in s:
        bigram_neg[w] += 1
        bigram_all[w] += 1
    
for s in train_token_pos:
    for w in s:
        bigram_pos[w] += 1
        bigram_all[w] += 1


In [6]:
vocab_list = []

with open('vocab_list.csv', 'rb') as csvfile:
    doc_reader = csv.reader(csvfile)
    vocab_list = next(doc_reader)

vocab_dict = {}
idx = 0
for w in vocab_list:
    vocab_dict[w] = idx
    idx += 1

print len(vocab_list)

8783


In [7]:
import numpy as np


alpha = 1.0
d = len(vocab_list)
p, q = p, q = np.ones(d) * alpha , np.ones(d) * alpha

for w in vocab_list:
    p[vocab_dict[w]] += bigram_pos[w]
    q[vocab_dict[w]] += bigram_neg[w]

p /= abs(p).sum()
q /= abs(q).sum()
r = np.log(p/q)
    

In [8]:
X_all = []
Y_all = []
for s in train_token_neg:
    vec = np.zeros(d)
    for w in s:
        if w in vocab_dict:
            vec[vocab_dict[w]] = r[vocab_dict[w]]

    X_all.append(vec)
    Y_all.append(-1)

for s in train_token_pos:
    vec = np.zeros(d)
    for w in s:
        if w in vocab_dict:
            vec[vocab_dict[w]] = r[vocab_dict[w]]

    X_all.append(vec)
    Y_all.append(1)


In [9]:
from sklearn.svm import LinearSVC
import pickle

with open('classifier/clf_svc.pickle', 'rb') as f:
    clf_svc_o = pickle.load(f)

print clf_svc_o.score(X_all[100000:110000]+X_all[400000:410000], Y_all[100000:110000]+Y_all[400000:410000])

0.95325


In [10]:
from sklearn.linear_model import LogisticRegression
with open('classifier/clf_lr.pickle', 'rb') as f:
    clf_lr_o = pickle.load(f)

print clf_lr_o.score(X_all[100000:110000]+X_all[400000:410000], Y_all[100000:110000]+Y_all[400000:410000])

0.9552


In [11]:
print r

[ 0.6766152   1.00147899  0.81081579 ... -0.76472057  0.91358675
 -2.81290957]


In [12]:
#MNB model
cnt_true = 0.0
cnt_all = len(X_all)
for i in range(len(X_all)):
    log_prob = np.sum(X_all[i])
    if((log_prob > 0 and Y_all[i] == 1) or (log_prob < 0 and Y_all[i] == -1)):
        cnt_true += 1
        
print "Accuracy: %f" % (cnt_true/cnt_all)
        
        
def eval_sent_MNB(s):
    vec = np.zeros(d)
    for w in s:
        if w in vocab_dict:
            vec[vocab_dict[w]] = r[vocab_dict[w]]
            
    return [np.sum(vec)]


    

Accuracy: 0.937251


In [42]:
def eval_sent_svm(clf, s):
    
    vec = np.zeros(d)
    for w in s:
        if w in vocab_dict:
            vec[vocab_dict[w]] = r[vocab_dict[w]]
        
    #print clf.predict([vec])
    return clf.decision_function([vec])

def eval_sent_lr(clf, s):
    
    vec = np.zeros(d)
    for w in s:
        if w in vocab_dict:
            vec[vocab_dict[w]] = r[vocab_dict[w]]
            
    return clf.predict_proba([vec])

In [83]:
print eval_sent_MNB(tokenize_std("worst", 2))
print eval_sent_svm(clf_svc_o, tokenize_std("worst", 2))
print eval_sent_lr(clf_lr_o, tokenize_std("worst", 2))

[-7.477022287621368]
[-2.12071489]
[[9.99728985e-01 2.71015263e-04]]


In [20]:
#load word2vec
from gensim import models
print('Loading models')
wv_model = models.Word2Vec.load('sentiment_train_0.model')


Loading models


In [64]:
res = wv_model.most_similar('lackluster',topn = 50)
for item in res:
    print(item[0]+","+str(item[1]))

unimpressive,0.770740509033
dreadful,0.770538389683
sub-par,0.763555645943
underwhelming,0.763401150703
horrendous,0.753042399883
subpar,0.750019311905
mediocre,0.729734063148
horrific,0.722833395004
lousy,0.720081806183
bizarre,0.716262221336
uninspired,0.713977336884
horrid,0.712019562721
aweful,0.711204051971
inconsistent,0.709859967232
marginal,0.708654463291
dismal,0.707518041134
substandard,0.706911206245
lacking,0.699179828167
forgettable,0.693775832653
terrible,0.693178057671
atrocious,0.68846821785
so-so,0.68160867691
appalling,0.680255115032
disappointing,0.674453496933
superb,0.669329166412
deplorable,0.669301390648
pathetic,0.664413452148
impersonal,0.662099838257
pitiful,0.654985785484
spotty,0.6481564641
absent,0.646654367447
horrible,0.644436895847
awful,0.644290208817
abysmal,0.640169501305
unbearable,0.639844179153
poor,0.63912320137
impeccable,0.633457005024
choppy,0.625445365906
questionable,0.621066808701
descent,0.619706690311
exquisite,0.619646370411
exceptional,0

In [102]:
import math
try_replace = ['JJ', 'JJR', 'JJS', 'RB', 'RBR', 'RBS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']


#retrieval model

def ret_words(s_token, w, i):
    #sent2vec
    sent_vec = np.zeros(128)
    for s in s_token:
        if s in wv_model.wv:
            sent_vec += wv_model.wv[s]
    
    sent_vec /= len(s_token)
    sent_vec = 0.3*sent_vec + 0.7*wv_model.wv[w]
    #retrieve
    ret = wv_model.similar_by_vector(sent_vec, topn=60, restrict_vocab=None)
    #print ret
    return ret
    
#delete words by contribution to sentiment
def sent_del(s_token, s_pos, eval_ori):
    s_len = len(s_token)
    del_tokens = []
    for i in range(s_len):
        if s_token[i] not in vocab_dict:
            continue
        if(s_pos[i][1] in try_replace):
            s_test = s_token[0:s_len]
            s_test[i] = "#"
            #eval_del = eval_sent_MNB(tokenize_std(" ".join(s_test), 2))
            eval_del = eval_sent_svm(clf_lr_o, tokenize_std(" ".join(s_test), 2))
            delta = eval_del[0] - eval_ori[0]
            #delta = eval_del[0][1] - eval_ori[0][1]
            if(delta > 1.0):
                print s_pos[i], eval_del[0] - eval_ori[0]
                del_tokens.append((i, s_token[i], s_pos[i][1]))
                
    return del_tokens

#insert into sentences with deleted words
def sent_ins_bi_dir(s_token, s_pos, del_tokens, eval_ori):
    s_len = len(s_token)
    s_new = s_token[0:s_len]
    for p in del_tokens:
        i = p[0]
        if(p[1] == "n't"):
            s_new[i] = ""
            continue
        
        ret = ret_words(s_token, p[1], i)
        max_gain = 0.0001
        to_insert = -1
        for k in range(len(ret)):
            if ret[k][0] == p[1]:
                continue
            pos_test = nltk.pos_tag([ret[k][0]])
            if(pos_test[0][1] == s_pos[i][1]):
                s_test = s_token[0:s_len]
                s_test[i] = ret[k][0]
                #eval_del = eval_sent_MNB(tokenize_std(" ".join(s_test), 2))
                eval_ins = eval_sent_svm(clf_lr_o, tokenize_std(" ".join(s_test), 2))
                delta = eval_ins[0] - eval_ori[0]
                #delta = eval_ins[0][1] - eval_ori[0][1]
                if(delta > max_gain):
                    max_gain = delta
                    to_insert = k
                
        if(to_insert > -1):
            s_new[i] = ret[to_insert][0]
            print "Edit by similarity"
        elif i == 0:
            s_new[i] = ""
        else:
            #Use bigram to insert a word
            max_prob = -1.0
            to_insert = -1
            for k in range(len(ret)):
                if ret[k][0] == p[1]:
                    continue
                test_prob = 0.001
                _bi = s_new[i-1]+"_"+ret[k][0]
                bi_ = ret[k][0]+"_"+s_new[i+1]
                _uni = s_new[i-1]
                uni_ = ret[k][0]
                
                if _uni in bigram_pos and uni_ in bigram_pos:
                    test_prob = float(bigram_pos[_uni])*float(bigram_pos[uni_])/100000.0
                else:
                    test_prob = 0.001
                    
                if _bi in bigram_pos and _uni in bigram_pos:
                    test_prob = float(bigram_pos[_bi])/float(bigram_pos[_uni])
                if bi_ in bigram_pos and uni_ in bigram_pos:
                    test_prob *= float(bigram_pos[bi_])/float(bigram_pos[uni_])    
                else:    
                    test_prob *= 0.001
            
                if test_prob > max_prob:
                    max_prob = test_prob
                    to_insert = k
            
            if(to_insert > -1):
                s_new[i] = ret[to_insert][0]
                print "Edit by language model"
            else:
                s_new[i] = ""
                        
    return s_new
            
        

In [104]:
num_rnds = 1
#try_replace = ['JJ', 'JJR', 'JJS', 'RB', 'RBR', 'RBS']

try_replace = ['JJ', 'JJR', 'JJS', 'RB', 'RBR', 'RBS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']

test_raw_neg = []
test_result = []
with open('data/yelp/sentiment.test.0', 'rb') as fp:
    test_raw_neg = fp.readlines()


for s in test_raw_neg:
    s_token = nltk.word_tokenize(s)
    s_pos = nltk.pos_tag(s_token)
    #eval_del = eval_sent_MNB(tokenize_std(s, 2))
    eval_ori = eval_sent_svm(clf_lr_o, tokenize_std(s, 2))
    del_tokens = sent_del(s_token, s_pos, eval_ori)
    for p in del_tokens:
        s_token[p[0]] = "#"
    #eval_ori = eval_sent_lr(clf_lr_o, tokenize_std(" ".join(s_token), 2))
    s_new = sent_ins_bi_dir(s_token, s_pos, del_tokens, eval_ori)
    for i in range(len(s_new)):
        if s_new[i] == "wo":
            s_new[i] = "will"
        elif s_new[i] == "ca":
            s_new[i] = "can"
            
    s_res = " ".join(s_new)
    print s, s_res
    test_result.append([s, s_res])

with open('test_bi_dir_lr.csv', 'wb') as csvfile:
    output_writer = csv.writer(csvfile)
    for row in test_result:
        output_writer.writerow(row)   
    

('changed', 'VBN') 2.610975437410394
Edit by similarity
ever since joes has changed hands it 's just gotten worse and worse .
ever since joes has improved hands it 's just gotten worse and worse .
('definitely', 'RB') 2.6259694961900184
('enough', 'RB') 4.003615411724258
('room', 'VB') 1.8815114764813483
Edit by similarity
Edit by similarity
Edit by language model
there is definitely not enough room in that part of the venue .
there is truly not pretty hotel in that part of the venue .
('basically', 'RB') 1.550279931527749
('tasted', 'VBN') 1.1654634188364383
('watered', 'VBN') 4.5614704061714475
Edit by similarity
Edit by language model
Edit by language model
so basically tasted watered down .
so pretty flavor calm down .
('said', 'VBD') 2.2558878230302026
Edit by similarity
she said she 'd be back and disappeared for a few minutes .
she figured she 'd be back and disappeared for a few minutes .
("n't", 'RB') 8.457612951192266
('believe', 'VB') 5.972042437498406
Edit by language model

In [ ]:
#The rest is not important

In [ ]:

def sent_ins_mono(s_token, s_pos, del_tokens, eval_ori):
    s_len = len(s_token)
    s_new = s_token[0:s_len]
    for p in del_tokens:
        i = p[0]
        if(p[1] == "n't"):
            s_new[i] = ""
            continue
        
        ret = ret_words(s_token, p[1], i)
        max_gain = 0.005
        to_insert = -1
        for k in range(len(ret)):
            pos_test = nltk.pos_tag([ret[k][0]])
            if(pos_test[0][1] == s_pos[i][1] and ret[k][1] > 0.005):
                s_test = s_token[0:s_len]
                s_test[i] = ret[k][0]
                eval_ins = eval_sent_lr(clf_lr_o, tokenize_std(" ".join(s_test), 2))
                delta = eval_ins[0][1] - eval_ori[0][1]
                if(delta > max_gain):
                    max_gain = delta*ret[k][1]
                    to_insert = k
                
        if(to_insert > -1):
            s_new[i] = ret[to_insert][0]
        elif i == 0:
            s_new[i] = ""
        else:
            #Use bigram to insert a word
            max_prob = 0.0000001
            to_insert = -1
            for k in range(len(ret)):
                test_prob = 0.0000001
                _bi = s_new[i-1]+"_"+ret[k][0]
                bi_ = ret[k][0]+s_new[i+1]
                _uni = s_new[i-1]
                if _bi in bigram_pos:
                    test_prob += float(bigram_pos[_bi])
                    
                if _uni in bigram_pos:
                    test_prob /= (float(bigram_pos[_uni])+1.0)
                else:
                    test_prob /= 1.0
                
                if test_prob > max_prob:
                    max_prob = test_prob
                    to_insert = k
        
            if(to_insert > -1):
                s_new[i] = ret[to_insert][0]
            else:
                s_new[i] = ""
                        
    return s_new

In [183]:
num_rnds = 1
#try_replace = ['JJ', 'JJR', 'JJS', 'RB', 'RBR', 'RBS']

try_replace = ['JJ', 'JJR', 'JJS', 'RB', 'RBR', 'RBS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']

for s in test_raw_neg[100:200]:
    
    s_token = nltk.word_tokenize(s)
    s_pos = nltk.pos_tag(s_token)
    eval_ori = eval_sent_lr(clf_lr_o, tokenize_std(s, 2))
    del_tokens = sent_del(s_token, s_pos, eval_ori)
    for p in del_tokens:
        s_token[p[0]] = "#"
    eval_ori = eval_sent_lr(clf_lr_o, tokenize_std(" ".join(s_token), 2))
    s_new = sent_ins_mono(s_token, s_pos, del_tokens, eval_ori)
    s_res = " ".join(s_new)
    print s, s_res
    

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [131]:
num_rnds = 1
try_replace = ['JJ', 'JJR', 'JJS', 'RB', 'RBR', 'RBS']

#try_replace = ['JJ', 'JJR', 'JJS', 'RB', 'RBR', 'RBS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
for s in train_raw_neg[3000:3010]:
    s_token = nltk.word_tokenize(s)
    s_pos = nltk.pos_tag(s_token)
    s_len = len(s_token)
    
    eval_ori = eval_sent_MNB(tokenize_std(s, 2))
    edit = 0
    for i in range(s_len):
        if(s_pos[i][1] in try_replace):
            s_test = s_token[0:s_len]
            s_test[i] = '#'
            eval_del = eval_sent_MNB(tokenize_std(" ".join(s_test), 2))
            if(eval_del[0] - eval_ori[0] > 1.0):
                
                edit = 1
                if s_token[i] not in vocab_dict:
                    continue
                print s_pos[i], eval_del[0] - eval_ori[0]
                ret = wv_model.most_similar(s_token[i],topn=20)
                max_gain_ins = 5.0
                to_insert = -1
                for k in range(len(ret)):
                    pos_test = nltk.pos_tag([ret[k][0]])
                    if(pos_test[0][1] == s_pos[i][1] and ret[k][1] > 0.5):
                        s_test_ins = s_token[0:s_len]
                        s_test_ins[i] = ret[k][0]
                        eval_ins = eval_sent_MNB(tokenize_std(" ".join(s_test_ins), 2))
                        dif_ins = eval_ins[0] - eval_ori[0]
                        if(dif_ins > max_gain_ins):
                            max_gain_ins = dif_ins
                            to_insert = k
                
                if(to_insert > -1):
                    s_token[i] = ret[to_insert][0]
                else:
                    s_token[i] = ""
                
        
    
    s_res = " ".join(s_token)
    if(edit == 1):
        print s, s_res
        
    #print res[to_insert][0], max_gain_ins

('horrible', 'JJ') 12.34938682941633
horrible service .
wonderful service .
('back', 'RB') 1.0040452015014338
('quickly', 'RB') 1.4043467773347107
bartender came back quickly with what looked like watered down lemonade .
bartender came   with what looked like watered down lemonade .
('ever', 'RB') 1.1866907608589532
worst margarita i have ever been served !
worst margarita i have  been served !
("n't", 'RB') 9.792309716462091
('even', 'RB') 14.711757090543378
did n't even get time under the warmer .
did   get time under the warmer .
("n't", 'RB') 5.275764773877574
('back', 'RB') 6.858047447884405
glad i tried this place but wo n't rush back .
glad i tried this place but wo not rush  .
('mediocre', 'JJ') 9.368369656535894
('poor', 'JJ') 24.43774378034813
service was mediocre but the food was poor .
service was marginal but the food was impeccable .


In [ ]:
for s in train_raw_neg[0:100]:
    
    
    for i in [n_gram-i for i in range(n_gram)]:
        for j in range(len(s)-i):
            tokens.append("_".join(s[j:j+i+1]))
            

In [68]:
test_pos = nltk.pos_tag(["noticeable"])
print test_pos

[('noticeable', 'JJ')]


In [98]:
nltk.help.upenn_tagset()

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or

In [40]:
ex = ["Hi", "There"]
print ex[0:1]

['Hi']
